In [1]:
# pip install numpy pandas requests

In [ ]:
import json
import requests
import numpy as np
import pandas as pd

/Users/kibeom/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [50]:
http_product_url = "https://api.neuralfin.ai"
productApiPath = "product/api/v1"
allfund_token  = "allfundsToken"

ALLFUND_PATH = f'{http_product_url}/{productApiPath}/funds'

In [63]:
def get_fund_catalog_data():

    result = requests.get(url=ALLFUND_PATH+'/catalog')
    content = json.loads(result.content)

    if content['status'] == 'success':
        data = content['data']['funds']
    else:
        raise 'Data cannot be fetched from API.'
    
    fund_table = pd.DataFrame(list(data))

    # extract company column to id and name of the company
    fund_table['company_id'] = fund_table['company'].apply(lambda x: x['allfunds_id'])
    fund_table['company_name'] = fund_table['company'].apply(lambda x: x['name'])

    # get important info
    fund_catalog = fund_table[['allfunds_id', 'isin','currency','company_id','company_name','product_status','last_updated_portfolio_date','created_at','updated_at']]
    return fund_catalog

In [121]:
# get overview given fund isin
def single_fund_overview(isin:str):
    assert isin[:2].isalpha, 'Invalid ISIN: First two-letter country code is unavailable.'
    
    overview_url = f'{http_product_url}/{productApiPath}/funds/{isin}/overview'
    response = requests.get(overview_url)

    if response.status_code != 200:
        raise ValueError("can't fetch data.")

    # extract data from response
    data = json.loads(response.content)['data']

    # convert it to dataframe
    df_indiv_overview = pd.DataFrame.from_dict(data, orient='index').T
    return df_indiv_overview

def dlifo_fund_overview(isin_codes: list[str]) -> pd.DataFrame:
    
    # run overview fetch individually.
    overview_data = [single_fund_overview(isin) for isin in isin_codes]
    overview_data = pd.concat(overview_data, axis=0)
    
    # select useful data
    interested_col = ['isin', 'name', 'fund_company', 'asset_class', 'subasset_class', 'category',  'inception_date', 'risk_reward_indicator', 'fund_benchmark', 'investment_objective', 'fund_aum', 'nav', 'aum_currency', ]
    selected_data = overview_data[interested_col]
    
    # extract key info from investment objective (english)
    selected_data['investment_objective'] = selected_data['investment_objective'].apply(lambda x: x['en']).values
    return selected_data

In [129]:
import datetime

str(datetime.date.today())

'2025-06-07'

In [169]:
from typing import Optional
def single_fund_navs(isin:str, since_date:str, until_date:Optional[str]=None) -> pd.DataFrame:
    assert isin[:2].isalpha, 'Invalid ISIN: First two-letter country code is unavailable.'
    
    if until_date is None:
        until_date = str(datetime.date.today())
    
    overview_url = f'{http_product_url}/{productApiPath}/funds/{isin}/close_prices'
    response = requests.get(overview_url, params={'since_date': since_date,'until_date': until_date})

    if response.status_code != 200:
        return "can't fetch data."
    
    # extract NAV data from response
    data = json.loads(response.content)['data']
    df_nav = pd.DataFrame(data['close_prices'])
    df_nav['isin'] = isin
    return df_nav[['isin', 'date', 'value']]


def dlifo_fund_navs(isin_codes: list[str], since_date:str, until_date:Optional[str]=None) -> pd.DataFrame:
    if until_date is None:
        until_date = str(datetime.date.today())
    
    # run nav fetch individually.
    nav_data = [single_fund_navs(isin, since_date, until_date) for isin in isin_codes]
    nav_data = pd.concat(nav_data, axis=0)
    
    # extract key info from investment objective (english)
    nav_data.columns = ['isin', 'date', 'close']
    return nav_data

In [122]:
# get list of fund and base info 
df_catalog = get_fund_catalog_data()

# get overview data for all DLiFO listed funds
df_overview = dlifo_fund_overview(df_catalog['isin'].tolist())

/var/folders/wf/_9l1644x31v3s39dpqrwmk9h0000gp/T/ipykernel_36898/3523335496.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['investment_objective'] = selected_data['investment_objective'].apply(lambda x: x['en']).values


In [170]:
df_nav = dlifo_fund_navs(df_catalog['isin'].tolist(), '2020-01-01')

In [195]:
def single_fund_performance(isin:str) -> pd.DataFrame:
    assert isin[:2].isalpha, 'Invalid ISIN: First two-letter country code is unavailable.'
    
    performance_url = f'{http_product_url}/{productApiPath}/funds/{isin}/performance'
    response = requests.get(performance_url)

    if response.status_code != 200:
        return "can't fetch data."
    
    # extract performance data from response
    data = json.loads(response.content)['data']['performance']
    
    # delete and add data to data dict
    data['isin'] = isin
    del data['quartiles'], data['quarterly_returns'], data['monthly_returns'], data['yearly_returns']
    return pd.DataFrame.from_dict(data, orient='index').T


def dlifo_fund_performance(isin_codes: list[str]) -> pd.DataFrame:
    # run nav fetch individually.
    performance_data = [single_fund_performance(isin) for isin in isin_codes]
    performance_data = pd.concat(performance_data, axis=0)
    return performance_data[['isin', 'inception', 'one_day', 'one_week', 'one_month', 'three_months', 'six_months', 'one_year', 'two_years', 'three_years', 'five_years', 'ten_years']]

In [174]:
isin_dummy = df_catalog['isin'][1]

In [196]:
df_performance = dlifo_fund_performance(df_catalog['isin'].tolist()) 

In [197]:
df_performance

,isin,inception,one_day,one_week,one_month,three_months,six_months,one_year,two_years,three_years,five_years,ten_years
0,LU1295555210,280.1,99.644255,101.119134,105.221638,104.049034,104.553938,113.354917,137.573674,145.204769,178.067387,NaN
0,LU0469268972,177.212398,101.081081,102.501563,106.276408,107.039938,107.390907,105.996347,121.653053,107.051546,134.79398,132.008432
0,IE00B6VH4D24,192.5809,100,100.494234,101.709045,101.20282,103.21489,109.564436,125.838061,121.031746,127.415144,163.978495
0,LU0258954444,151.756523,100.025543,100.212488,102.133163,100.903606,101.800787,105.374858,116.541815,121.329999,137.934574,149.337191
0,LU0258954014,226.25,100.035372,100.217045,102.135247,100.914362,101.804356,105.384508,116.557622,121.326684,137.948906,151.085142
0,IE00B2Q1FD82,167.72,100.467234,100.353019,100.951005,104.667998,104.726819,110.197109,110.494763,109.951488,105.643739,122.004801
0,LU0334857355,181.8,100.94392,101.564246,103.589744,102.134831,105.452436,113.981191,121.442886,115.063291,97.32334,124.948454
0,LU0334857199,181.892622,100.999412,101.596688,103.618818,102.140309,105.463475,114.014235,121.427477,115.069131,97.347296,124.987339
0,LU1084869962,313.502,100.179267,101.059913,104.637044,103.985246,102.709751,113.597559,143.771984,144.038189,161.539849,272.847694
0,IE00B91X6F72,167.460681,99.800995,100.501002,101.125479,100.457094,102.328964,106.376802,114.51357,116.130432,118.292079,144.858294
